# MISP & PyMISP

## Documentation

- API Specs: https://www.misp-project.org/openapi/
- Cheatsheet: https://www.misp-project.org/misp-training/cheatsheet.pdf
- MISP REST API: https://github.com/MISP/misp-training/blob/main/a.7-rest-API/Training%20-%20Using%20the%20API%20in%20MISP.ipynb
    - query: https://github.com/MISP/misp-training/blob/main/a.7-rest-API/query-misp.ipynb
- PyMISP: https://github.com/MISP/PyMISP/blob/main/docs/tutorial/FullOverview.ipynb

## Exercises
### PyMISP

#### PyMISP bootstrap


In [ ]:
import pymisp
import urllib3
import getpass

MISP_BASEURL = "https://training.misp-community.org"
MISP_API_KEY = getpass.getpass("Enter your MISP AuthKey:")

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

misp = pymisp.PyMISP(
    MISP_BASEURL,
    MISP_API_KEY,
    ssl=False # Disable SSL verification
)
print(f"Connected to MISP {misp.root_url} running version: {misp.version['version']}")

#### 1.0. Get your user information
Get the `email`, `user_id` and `role` of your user.
- *Hint*: related API endpoint: `/users/view/me` 

In [ ]:
user, role, settings = misp.get_user(pythonify=True, expanded=True)
print(f"User ID: {user.id}, Email: {user.email}, Role: {role.name}")

#### 1.1. Get Event with UUID `83a7add9-76d7-47ef-9f4b-ebd07fbe880d`
- *Hint*: related API endpoint: `/events/view/[uuid]`

In [ ]:
event = misp.get_event("83a7add9-76d7-47ef-9f4b-ebd07fbe880d", pythonify=True)
print(event.info, event.uuid)

#### 1.2. Get all Events related to the following threat actor
- Threat actor: `Deadeye Jackal`
- *Hint*: `misp-galaxy:threat-actor="Deadeye Jackal"`
- *Hint*: You can use either `/events/index` or `/events/restSearch`

In [ ]:
events = misp.search("events", tag="misp-galaxy:threat-actor=\"Deadeye Jackal\"", pythonify=True)
print(events)

#### 1.3. Get all URLs
- Get all Attributes of type `url` meant for **protective tools**
- *Hint*: You can use `/attributes/restSearch`
- *Hint*: Protective tool means `to_ids` set to `1`

In [ ]:
urls = misp.search("attributes", type_attribute=["url"], pythonify=True)
for url in urls:
    print(url.value)

#### 1.4. Get all Attributes that were **published** in the **past 7 days** and are currently in the **published state**
- *Hint*: You can use `/attributes/restSearch`

In [ ]:
ips = misp.search("attributes", type_attribute=["ip-src", "ip-dst"], publish_timestamp="7d", published=True, pythonify=True)
for ip in ips:
    print(ip.value)

#### 1.5. Get Events targetting the Government sector abusing T1154 MITRE Technique 
- *Hint*: Tag for Government sector: `misp-galaxy:sector="Government, Administration"`
- *Hint*: Tag for MITRE Attack Pattern T1154: `misp-galaxy:mitre-attack-pattern="Compromise Client Software Binary - T1554"`

In [ ]:
events = misp.search("events", tag={
        "AND": [
            "misp-galaxy:sector=\"Government, Administration\"",
            "misp-galaxy:mitre-attack-pattern=\"Compromise Client Software Binary - T1554\""
        ]
    }, pythonify=True)
print(events)